In [ ]:
#| default_exp code-assistant

# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from ddgs import DDGS
from fastcore.all import *
from litesearch import *
from IPython import get_ipython
import json
from lisette import *
import numpy as np
import os
import sys
from toolslm.funccall import *

Let's set up the database and create the necessary tables. packages will store package metadata and store will store code snippets with their embeddings.

In [ ]:
#| export
def mk_store(pth='code.db'):
	'Setup litesearch and create needed tales for code storage.'
	db = setup_db(pth)
	store = db.mk_store()
	if 'package' not in store.c: store.add_column('package', str)
	packages = db.t.packages.create(name=str, version=str, summary=str, uploaded_at=float, pk=['name'], defaults=dict(uploaded_at='CURRENT_TIMESTAMP'), not_null=['name'], if_not_exists=True)
	return db, store, packages

In [ ]:
db,store,packages = mk_store()

In [ ]:
len(store(where='embedding is NULL'))

# Ingestion Pipeline

## Chunk Packages

Let's create a simple code ingestion pipeline which ingests functions, classes, and global variables from python packages into a litesearch database for searching. We'll be using ast module to parse python files and extract relevant code snippets.

In [ ]:
#| export
def code2chunks(pkg,p):
	import ast
	from ast import get_source_segment as gs
	txt=Path(p).read_text(encoding='utf-8')
	tree=ast.parse(txt)
	[setattr(c,'parent',n) for n in ast.walk(tree) for c in ast.iter_child_nodes(n)]
	def n2c(n): return dict(content=gs(txt, n).strip(), metadata=p, package=pkg, uploaded_at=Path(p).stat().st_mtime)
	def is_func(n): return isinstance(n,(ast.FunctionDef,ast.AsyncFunctionDef,ast.ClassDef))
	def is_assign(n): return isinstance(n, (ast.Assign, ast.AnnAssign)) and n.value
	def is_p_mod(n): return getattr(getattr(n,'parent',None),'__class__',None) == ast.Module
	def is_global(n): return False if not is_assign(n) else True if is_p_mod(n) else False
	def is_allowed(n): return is_func(n) or is_global(n)
	return L(ast.walk(tree)).filter(is_allowed).map(n2c)

The main ingestion function which takes a list of package names, extracts python files from them, parses the files to extract functions, classes, and global variables, and stores them in the database.

In [ ]:
#| export
py_dir_skip_re=r'(^tests?$|^__pycache__$|^\.eggs$|^\.mypy_cache$|^\.tox$|^examples?$|^docs?$|^build$|^dist$|^\.git$|^\.ipynb_checkpoints$)'
py_file_skip_re=r'(^__init__\.py$|^setup\.py$|^conftest\.py$|^test_.*\.py$|^tests?\.py$|^.*_test\.py$)'
py_glob, skip_py_glob = '*.py', '_*'

@delegates(globtastic)
def pkg2files(pkg:str,								# package name
              file_glob:str=py_glob,				# file glob to match
              skip_file_glob:str=skip_py_glob,		# file glob to skip
              skip_file_re=py_file_skip_re, 		# regex to skip files
              skip_folder_re=py_dir_skip_re, 		# regex to skip folders
			  **kwargs								# additional args to pass to globtastic
)->L:
	'Return list of python files in a package excluding tests and setup files.'
	return globtastic(Path(spec.origin).parent,file_glob=file_glob,skip_file_glob=skip_file_glob,
        folder_re=pkg, skip_folder_re=skip_folder_re, skip_file_re=skip_file_re, **kwargs)

def ingest(pkgs=list):
    def _(pkg):
        from importlib.metadata import version as v, metadata as meta
        if packages(select='name', where=f'name={pkg!r} and version={v(pkg)!r}'): return
        packages.upsert(dict(name=pkg, version=v(pkg)), summary=meta(pkg)['Summary'], pk='name')
        store.insert_all(pkg2files(pkg).map(lambda p: code2chunks(pkg,p)).concat(), upsert=True)
    return L(pkgs).map(_)

def pkgs(nms:list=None, full=False):
	from importlib.util import find_spec as fs
	from importlib.metadata import distributions as dists, distribution as dist
	not_stdlib = lambda d: d.metadata.get('Author-email') not in ('Python', None)
	pkgs = L(dists()) if full else L(nms).filter(fs).map(dist)
	return pkgs.filter(not_stdlib).map(lambda d: d.metadata['Name'])

We'll use modern-bert-base onnx model for embedding the code snippets. We'll ingest the chunks first and then embed them. You can do both together. I like to separate it so that I can add more columns and test with different embedding models later. eg: `store.add_column('test_embedding', bytes)` and then `embed_docs(col='test_embedding', f=other_model.encode_document)`

In [ ]:
pl = ['fastcore', 'fastlite', 'lisette', 'toolslm', 'chonkie', 'model2vec']
ingest(pkgs(pl))

## Encoding Setup

FasstEncode is a simple ONNX based embedding model wrapper which can work with most onnx models with a huggingface tokenizer.

In [ ]:
m1 = FastEncode(modernbert, md='models/%s' % modernbert.model)
doc_emb, query_emb = m1.encode_document, m1.encode_query

## Embedding code chunks

The main embedding function which embeds all documents in the store table using the provided embedding function.

In [ ]:
#| export
def embed_docs(st=None, f=None, col='embedding', sz=100, reembed=False):
	'Embed all documents in a table using emb_f'
	if not (st and f): return
	n = len(st(where='embedding is NULL' if reembed else None))
	for offset in range(0, n, sz):
		batch=st(where=f'{col} is NULL', limit=sz, offset=offset)
		c = [b['content'] for b in batch if b['content'].strip()]
		if not c: continue
		for e,b in zip(f(c),batch): b[col] = e.tobytes()
		st.upsert_all(batch, pk='id')

In [ ]:
log = lambda: print('Docs without embedding: ', len(store(select='id',where='embedding is NULL')),'\n','-'*20)
log();embed_docs(st=store, f=doc_emb);log() # using modern-bert-base onnx model. let's make sure all chunks are embedded

# Coding Assistant

## Coding Assistant Tools

Some helper functions and tools for our coding assistant.

In [ ]:
#| export
@timed_cache(3600)
def get_pkgs():
    'Return list of ingested package names.'
    return [p['name'] for p in packages(select='name')]

def code_search(q:str,               	# query to search
                emb_q:str=None,     	# query to embed. If None, use q
				emb_f= query_emb, # embedding function
                wide:bool=False,    	# whether to use wide search
                **kw					# additional args to pass to db.search
):
	'Code search through the database to find relevant code snippets.'
	emb = emb_f(emb_q if emb_q else q)
	available = L(set(q.split(' ')).intersection(get_pkgs()))
	wh = f'package in ({','.join(available.map(repr))})' if available else None
	kw['where'] = wh if 'where' not in kw else f"({kw['where']}) AND {wh}" if wh else kw['where']
	return db.search(pre(q, wide=wide), emb.tobytes(), **kw)

def websearch(q: str,               # query to search
              top_k: int = 10,      # number of top results to return
              ):
	'Web search results reranked with flashrank'
	res = dict2obj(DDGS().text(q, max_results=top_k))
	return json.dumps([dict(text=r.body, id=r.href, meta=dict(title=r.title)) for r in res])

def rag(q: str,	# query string
        emb_q: str = None, # query to embed. If None, use q
        top_k: int = 5, # number of top results to return
        wide: bool = False, # whether to use wide search
        web: bool = False # whether to include web search results
) -> str:
    'Search indexed code for relevant chunks. Returns structured results.'
    r = code_search(q, emb_q=emb_q if emb_q else q, lim=top_k, columns=['content', 'metadata'], wide=wide)
    if web: r += websearch(q, top_k=top_k // 2)  # Balance
    return json.dumps(dict(query=q, results=r, top_k=top_k))  # Or dict if no Pydantic


## Code Execution Tool

In [ ]:
#| export
def run_code(code:str, 	  # code to run
             strict=True # whether to run in strict mode
             )->str:
	'''Run code in python interpreter'''
	if not strict: python(code, globals())
	return python(code)

def get_globals():
	'Return current global variables.'
	return globals().keys()

#### Syntactic Sugar

In [ ]:
def ins(code: str):
	'Insert code into the next cell of the current notebook.'
	ip = get_ipython()
	if not ip: return
	ip.set_next_input(code.strip() + "\n", replace=False)

## LLM Chat with Tools Setup. Let's not call it an Agent. Maybe an Augmented LLM Chat?

In [ ]:
#| export
sp= f'''You are a razor-sharp Python coding assistant with perfect knowledge of fastcore, fastlite, lisette, toolslm, chonkie, model2vec and every package indexed in your RAG database.

Your ONLY job: answer with concise, working, copy-pasteable code. No essays. No apologies.
Available packages: {packages(select='name,summary')}.
TOOLS (in exact order you MUST follow):
1. rag(q: str, emb_q: str | None, top_k: int = 7) → ALWAYS call this first on every code question.
   - Use natural language q for FTS5, Do not pass the user query as is to q. Clean the query first. Add package names if teh query matches any ingested package. Use the summary of packages to help you.
   - Craft the right fts5 to get the best response.
   - Craft a precise emb_q for semantic search if needed.
   - top_k=5–10. Never more unless explicitly asked.
2. websearch(q) → ONLY if rag returned <3 useful chunks or the question is about unindexed packages.
3. run_code(code: str) → MANDATORY: execute the final example before replying. If it fails, fix and retry.
4. get_globals() → MANDATORY: call before any code that uses variables. Never clash with user namespace.

RESPONSE RULES — NON-NEGOTIABLE:
- Step 1 is internal thinking only. Never show it unless it contains a tool call.
- ALWAYS call rag() first. No exceptions for "simple" questions.
- Quote the most relevant source chunk verbatim (with path comment).
- Synthesize → minimal explanation → final runnable example.
- Final answer MUST be a single ```python code block. Nothing after it except optional one-sentence note.
- Judge the answer and If the answer is not good enough, research and reply with improved code.
- Use unique variable names (e.g. _result, _df, _items, _chat) unless user explicitly reuses theirs. Make sure you do not clash with user namespace.
- If run_code fails → fix silently and retry until it passes.
- Keep total response ≤ 250 words.

MEMORY: You remember every past example in this conversation. Reuse and refine them when relevant.

FAILURE IS NOT AN OPTION.
Be brutal. If the user’s idea is dumb, say so and give the right way.
  '''
chat=Chat('gpt-4.1', sp, tools=[rag, websearch, run_code, get_globals])
c=bind(chat, max_steps=8, return_all=True, max_tokens=10000)

syntactic sugar to format the last response to show code, tool name, code executed and code result.

In [ ]:
@patch
def fmt_res(self:Chat, copy:bool=True):
	'Format the last response to show code, tool name, code executed and code result.'
	import pyperclip
	lm,tr,ltc=self.hist[-1].content, self.hist[-2], self.hist[-3].tool_calls
	code = lm.split('```python')[-1].split('```')[0].strip()
	tn,ce,cr= tr['name'], json.loads(dict2obj(ltc[-1]).function.arguments)['code'], tr['content']
	if copy: pyperclip.copy(code); print('Code copied to clipboard!')
	return AttrDict(code=code, tool_name=tn, code_executed=ce, code_result=cr)

Let's try some code generation tasks now.

In [ ]:
r=c('parallel processing of a list in fastcore');print(chat.fmt_res().code)

In [ ]:
r=c('running a model inference using litellm');print(chat.fmt_res().code)

In [ ]:
code = '''
 for i, cell in enumerate(nb.cells):
        if cell.cell_type == 'code' and cell.source.strip() == src.strip():
            idx = i
            break'''
r=c(['I want to make this code more efficient using fastcore utilities. Rewrite it.', code]); print(chat.fmt_res().code)

We can use the `ai` function to insert code into the next cell of the current notebook and run it if we want to.

This time let's actually ask it to do things in the workspace. You're basically putting the agent to work on pointed tasks.

In [ ]:
r=c('how can I distill a model using model2vec and save it locally on a file to reuse later. take the example of nomic-ai/modernbert-embed-base');print(chat.fmt_res().code)

# Export Notebook

In [ ]:
#| hide
from nbdev.export import nb_export; nb_export('02_tool_use.ipynb')